In [1]:
from google.colab import files

In [2]:
!pip -q install kaggle pyspark pyarrow

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{\n  "username": "vadimsokol",\n  "key": "KGAT_1943f2f81f04e39172723dd31c2ac3c0"\n}\n'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets list | head


ref                                                                 title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
saidaminsaidaxmadov/chocolate-sales                                 Chocolate Sales                                         468320  2026-01-04 14:23:35.490000          10551        172  1.0              
jayjoshi37/customer-subscription-churn-and-usage-patterns           Customer Subscription Churn and Usage Patterns           34246  2026-01-27 13:53:52.857000            655         23  1.0              
vishardmehta/indian-engineering-college-placement-dataset           Indian Engineering College Placement Dataset            137603  2026-01-24 15:23:40.150000           1622         44

In [ ]:
DATASET = "mohamedbakhet/amazon-books-reviews"

!mkdir -p /content/data
!kaggle datasets download -d {DATASET} -p /content/data --unzip

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0


In [7]:
!ls -lah /content/data


total 2.9G
drwxr-xr-x 2 root root 4.0K Feb  1 11:32 .
drwxr-xr-x 1 root root 4.0K Feb  1 11:31 ..
-rw-r--r-- 1 root root 173M Feb  1 11:32 books_data.csv
-rw-r--r-- 1 root root 2.7G Feb  1 11:32 Books_rating.csv


In [8]:
DATA_DIR = "/content/data"
RATINGS_CSV = f"{DATA_DIR}/Books_rating.csv"
BOOKS_CSV = f"{DATA_DIR}/books_data.csv"


USE_FULL_DATA = False # just to save runtime
# 5 percent sample,
# TODO: try 10% later
SAMPLE_FRAC = 0.05
SEED = 12345




In [9]:
# Start the SPark session

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("amd-recsys-books")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")


In [10]:
# Load book ratings into Spark:
from pyspark.sql import functions as F

ratings = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(RATINGS_CSV)
)

ratings.printSchema()
ratings.show(5, truncate=False)


root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

+----------+------------------------------+-----+--------------+--------------------------------------+------------------+------------+-----------+-----------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# check the columns to understand what's inside

cols = {c.lower(): c for c in ratings.columns}
cols

{'id': 'Id',
 'title': 'Title',
 'price': 'Price',
 'user_id': 'User_id',
 'profilename': 'profileName',
 'review/helpfulness': 'review/helpfulness',
 'review/score': 'review/score',
 'review/time': 'review/time',
 'review/summary': 'review/summary',
 'review/text': 'review/text'}